In [12]:
!pip install ../ --upgrade

Processing /home/jovyan/work
  Found existing installation: rsplib 0.2.4
    Uninstalling rsplib-0.2.4:
      Successfully uninstalled rsplib-0.2.4
  Running setup.py install for rsplib ... done


In [14]:
from rsplib.experiments import Experiment, ExperimentExecution, Report
from rsplib.experiments.workflow import execute, deploy
from rsplib.processing.consumer.query import *

In [15]:
e = Experiment()

qname = "Q1"
e.set_duration(15,"minutes")
e.add_engine('http://csparql.westeurope.cloudapp.azure.com', 80, Dialects.CSPARQL)

q = e.add_query(qname, "query", Dialects.CSPARQL)
q.set_select_clause("?obId1 ?obId2 ?v1 ?v2")
q.set_where_clause("{?p1   a <http://www.insight-centre.org/citytraffic#CongestionLevel>.?p2   a <http://www.insight-centre.org/citytraffic#CongestionLevel>. {?obId1 <http://purl.oclc.org/NET/ssnx/ssn#observedProperty> ?p1.?obId1 <http://purl.oclc.org/NET/sao/hasValue> ?v1.?obId1 <http://purl.oclc.org/NET/ssnx/ssn#observedBy> <http://www.insight-centre.org/dataset/SampleEventService#AarhusTrafficData182955>.}{?obId2 <http://purl.oclc.org/NET/ssnx/ssn#observedProperty> ?p2.?obId2 <http://purl.oclc.org/NET/sao/hasValue> ?v2.?obId2 <http://purl.oclc.org/NET/ssnx/ssn#observedBy> <http://www.insight-centre.org/dataset/SampleEventService#AarhusTrafficData158505>.}}")
e.add_windowed_stream(qname,"AarhusTrafficData182955", "http://triplewave1.westeurope.cloudapp.azure.com:4000/sgraph", '3s','1s' )
e.add_windowed_stream(qname,"AarhusTrafficData158505", "http://triplewave1.westeurope.cloudapp.azure.com:4001/sgraph", '3s','1s' )

e.add_graph(qname, "SensorRepository", "https://rsplab.blob.core.windows.net/static/SensorRepository.rdf", serialization="RDF/XML", default='true')
#e.add_graph(qname, "AarhusLibraryEvents", "https://rsplab.blob.core.windows.net/static/AarhusLibraryEvents.rdf", serialization="RDF/XML", default='true')



In [16]:
e.__dict__()

{'duration': {'time': 15, 'unit': 'minutes'},
 'engine': {'dialect': 'CSPARQL',
  'host': 'http://csparql.westeurope.cloudapp.azure.com',
  'port': 80},
 'graphs': [{'serialization': 'RDF/XML', 'name': 'SensorRepository', 'default': 'true', 'location': 'https://rsplab.blob.core.windows.net/static/SensorRepository.rdf'}],
 'metadata': {},
 'queries': [{'type': 'query', 'dialect': 'CSPARQL', 'structure': {'having': '', 'where_clause': {'unnamed': [], 'named': [], 'default': ['{?p1   a <http://www.insight-centre.org/citytraffic#CongestionLevel>.?p2   a <http://www.insight-centre.org/citytraffic#CongestionLevel>. {?obId1 <http://purl.oclc.org/NET/ssnx/ssn#observedProperty> ?p1.?obId1 <http://purl.oclc.org/NET/sao/hasValue> ?v1.?obId1 <http://purl.oclc.org/NET/ssnx/ssn#observedBy> <http://www.insight-centre.org/dataset/SampleEventService#AarhusTrafficData182955>.}{?obId2 <http://purl.oclc.org/NET/ssnx/ssn#observedProperty> ?p2.?obId2 <http://purl.oclc.org/NET/sao/hasValue> ?v2.?obId2 <http:

In [17]:
ex = deploy(e)

{"name":"csparql","host":"csparql/","runUUID":"ae4c1e28-35b4-11e7-8ce9-000d3a27a904","port":8182,"empty_results":false,"inference":false,"timestam_function":false,"backloop":true,"num_streams":0,"num_datasets":0,"num_queries":0}
Registering static sources: https://rsplab.blob.core.windows.net/static/SensorRepository.rdf
"Dataset SensorRepository succesfully registered with IRI https://rsplab.blob.core.windows.net/static/SensorRepository.rdf"
Registering stream: AarhusTrafficData182955
"Stream AarhusTrafficData182955 succesfully registered with IRI http://triplewave1.westeurope.cloudapp.azure.com:4000/sgraph"
Registering stream: AarhusTrafficData158505
"Stream AarhusTrafficData158505 succesfully registered with IRI http://triplewave1.westeurope.cloudapp.azure.com:4001/sgraph"
Registering query Q1 
{'queryBody': 'REGISTER QUERY Q1 AS SELECT ?obId1 ?obId2 ?v1 ?v2 FROM STREAM <AarhusTrafficData182955> [RANGE 3s STEP 1s]\n FROM STREAM <AarhusTrafficData158505> [RANGE 3s STEP 1s]\nWHERE {{?p

In [18]:
ex.__dict__()

{'Execution': {'D': ['Dataset SensorRepository succesfully registered with IRI https://rsplab.blob.core.windows.net/static/SensorRepository.rdf'],
  'E': {'backloop': True,
   'empty_results': False,
   'host': 'csparql/',
   'inference': False,
   'name': 'csparql',
   'num_datasets': 0,
   'num_queries': 0,
   'num_streams': 0,
   'port': 8182,
   'runUUID': 'ae4c1e28-35b4-11e7-8ce9-000d3a27a904',
   'timestam_function': False},
  'K': None,
  'O': [{'Q1': {'@context': {'generatedAt': {'@id': 'http://www.w3.org/ns/prov#generatedAtTime',
       '@type': 'http://www.w3.org/2001/XMLSchema#dateTime'},
      'sld': 'http://streamreasoning.org/ontologies/SLD4TripleWave#'},
     '@id': 'default',
     '@type': 'sld:sGraph',
     'sld:streamLocation': 'ws://csparql.westeurope.cloudapp.azure.com:9101/Q1/results'}}],
  'Q': [{'id': 'Q1', 'status': 'RUNNING', 'type': 'QUERY'}],
  'R': None,
  'S': ['Stream AarhusTrafficData182955 succesfully registered with IRI http://triplewave1.westeurope.clo

In [19]:
execute(ex)

TypeError: a float is required

In [ ]:
ex.__dict__()

 <iframe src="http://52.178.103.179:3000/dashboard-solo/db/csparql?orgId=1&from=1494443406734&to=1494444306734&panelId=1&theme=light" width="450" height="200" frameborder="0"></iframe>

In [25]:
from IPython.display import IFrame
IFrame('http://52.178.103.179:3000/dashboard/db/csparql?orgId=1&from=now-30m&to=now', width=700, height=350)

In [1]:
!pip install pygrafana

  Could not find a version that satisfies the requirement pygrafana (from versions: )
No matching distribution found for pygrafana


In [1]:
from pygrafana.dashboard import Target, GraphPanel, Row, Dashboard
from pygrafana.api import Connection

SyntaxError: Missing parentheses in call to 'print' (dashboard.py, line 50)